### 多级模型 Multi-level models（可变截距）
***
这是一个如何使用 python 和 `pymer4` 包拟合 MLM 模型的示例。Pymer4 是 R 中著名的 `lme4` 包的 python 版本，其文档可在以下位置找到：https://eshinjolly.com/pymer4/


#### 安装

安装 `pymer4` 很麻烦。

`pymer4` 需要 R 环境。如果您的计算机没有 R，请从 [这里](https://www.r-project.org) 安装一个。安装 R 后，您还需要安装 `lme4` 和 `lmerTest`。打开您的 R 并将这两个包安装为

`install.packages("lme4")`

`install.packages("lmerTest")`

然后返回 python 安装 `pymer4`，方法为
`pip install pymer4`

In [1]:
#pip install pymer4

In [2]:
import warnings

import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import libpysal as ps
from libpysal.weights import Queen
from esda.moran import Moran

我们将使用相同的投票数据来演示 MLM。数据略有不同，因为它有一个额外的列，表示县所属州的州长。

加载投票数据集

In [3]:
voting = pd.read_csv('https://raw.github.com/Ziqi-Li/gis5122/master/data/voting_2020_with_gov.csv')

#voting[['median_income']] = voting[['median_income']]/10000

In [4]:
voting['is_dem_gov']= (voting.party == 'democrat')

In [5]:
shp = gpd.read_file("https://raw.github.com/Ziqi-Li/gis5122/master/data/us_counties.geojson")

In [6]:
#Merge the shapefile with the voting data by the common county_id
shp_voting = shp.merge(voting, on ="county_id")

shp_voting_df = shp_voting.drop(columns='geometry')

#Dissolve the counties to obtain boundary of states, used for mapping
state = shp_voting.dissolve(by='STATEFP').geometry.boundary

### 拟合 MLM 模型（根据州改变截距）

让我们首先看一下以下两两比较：
- 加利福尼亚州与马萨诸塞州
- 加利福尼亚州与德克萨斯州
请注意，州 FIPS 代码为：
CA:06;MA:25;TX:48

In [7]:
ca_ma = shp_voting[shp_voting['state'].isin([6, 25])]

ca_tx = shp_voting[shp_voting['state'].isin([6, 48])]

`Lmer()` 采用遵循`y ~ X` 格式的公式。1 表示添加截距，`(1|state)` 允许截距在不同状态之间变化。

California vs. Texas

In [8]:
warnings.filterwarnings('ignore')

from pymer4.models import Lmer

model = Lmer('new_pct_dem ~ 1 + (1|state)', data=ca_tx)

model.fit()

Linear mixed model fit by REML [’lmerMod’]
Formula: new_pct_dem~1+(1|state)

Family: gaussian	 Inference: parametric

Number of observations: 312	 Groups: {'state': 2.0}

Log-likelihood: -1278.940 	 AIC: 2563.881

Random effects:

                 Name      Var     Std
state     (Intercept)  458.854  21.421
Residual               210.864  14.521

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),39.981,10.222,69.74,15.184,1.0,2.633,0.231,


从摘要中可以看出：

- 有 312 个观测值（加利福尼亚州的县总数 + 德克萨斯州的县总数）
- 有 2 个组
- 该模型的 AIC 为 2563.881
- 州级随机效应的估计方差为 458.854，标准差为 21.421
- 残差的估计方差为 210.864，标准差为 14.521。
- 方差分割系数 (VPC) = 458.854/(458.854+210.864) = 68.5%。这意味着投票偏好中 68.5% 的方差可归因于县属于两个独立州的事实。
- 对固定效应的解释与我们在线性回归中的解释相同。没有预测因子，这里只有一个截距，截距估计值为 39.981，并且不具有统计学意义（p 值 = 0.231 > 0.05）。



California vs. Mass.

In [9]:
warnings.filterwarnings('ignore')

model2 = Lmer('new_pct_dem ~ 1 + (1|state)', data=ca_ma)

model2.fit()

Linear mixed model fit by REML [’lmerMod’]
Formula: new_pct_dem~1+(1|state)

Family: gaussian	 Inference: parametric

Number of observations: 72	 Groups: {'state': 2.0}

Log-likelihood: -293.063 	 AIC: 592.127

Random effects:

                 Name      Var     Std
state     (Intercept)   77.881   8.825
Residual               205.493  14.335

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),61.384,48.484,74.284,6.582,0.984,9.327,0.07,.


### 针对所有状态改变截距模型

In [10]:
warnings.filterwarnings('ignore')

model_all_states_1 = Lmer('new_pct_dem ~ 1 + (1|state)', data=shp_voting_df)

model_all_states_1.fit()

Linear mixed model fit by REML [’lmerMod’]
Formula: new_pct_dem~1+(1|state)

Family: gaussian	 Inference: parametric

Number of observations: 3102	 Groups: {'state': 47.0}

Log-likelihood: -12510.156 	 AIC: 25026.312

Random effects:

                 Name      Var     Std
state     (Intercept)  134.233  11.586
Residual               176.567  13.288

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),37.507,34.129,40.885,1.724,43.861,21.761,0.0,***


What is VPC in this case?

### 现在我们添加一些县级（个人）变量

这里我们将 `pct_bach` 纳入模型作为固定效应

In [11]:
warnings.filterwarnings('ignore')

model_all_states_2 = Lmer('new_pct_dem ~ 1 + pct_bach + (1|state)', data=shp_voting_df)

model_all_states_2.fit()

Linear mixed model fit by REML [’lmerMod’]
Formula: new_pct_dem~1+pct_bach+(1|state)

Family: gaussian	 Inference: parametric

Number of observations: 3102	 Groups: {'state': 47.0}

Log-likelihood: -12069.178 	 AIC: 24146.356

Random effects:

                 Name      Var     Std
state     (Intercept)   76.666   8.756
Residual               133.186  11.541

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),18.930,16.125,21.735,1.431,62.434,13.226,0.0,***
pct_bach,0.776,0.728,0.823,0.024,3097.039,32.086,0.0,***


模型 AIC 随着添加此 `pct_bach` 变量而下降。对此变量的解释是：在考虑州级差异后，`pct_bach` 的估计值为 0.776，p 值较低。在全国范围内，在其他因素保持不变的情况下，`pct_bach` 增加 1% 将导致投票给 DEM 的人增加 `0.776%`。

### 现在我们添加一个州级（组）变量

这里我们将`is_dem_gov`作为固定效应纳入模型

In [12]:
warnings.filterwarnings('ignore')

model_all_states_3 = Lmer('new_pct_dem ~ 1 + pct_bach + is_dem_gov + (1|state)', data=shp_voting_df)

model_all_states_3.fit()

Linear mixed model fit by REML [’lmerMod’]
Formula: new_pct_dem~1+pct_bach+is_dem_gov+(1|state)

Family: gaussian	 Inference: parametric

Number of observations: 3102	 Groups: {'state': 47.0}

Log-likelihood: -12065.543 	 AIC: 24141.087

Random effects:

                 Name      Var     Std
state     (Intercept)   72.165   8.495
Residual               133.190  11.541

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),16.666,13.087,20.244,1.826,51.528,9.127,0.000,***
pct_bach,0.775,0.728,0.823,0.024,3096.963,32.081,0.000,***
is_dem_govTRUE,4.864,-0.138,9.867,2.552,43.097,1.906,0.063,.


考虑到州级差异后，“is_dem_gov”的估计值为 4.864，p 值较高（0.063 > 0.05）。这意味着，在全国范围内，如果一个州有一位民主党州长，将为 DEM 带来 4.86% 的选票，然而，考虑到目前的数据和模型，这种影响在 0.05 的水平上并不具有统计学意义。